## Introduction to `robotoc` 
# 2: Robot  Models

## 本章の目的
URDFからロボット (`robotoc::Robot`) モデルを作成することが目的です.  
ここで， `robotoc::Robot`は [`Pinocchio`](https://github.com/stack-of-tasks/pinocchio) のAPIをラップしています.  
より深く知りたい方は， [`Pinocchio` の documentation](https://stack-of-tasks.github.io/pinocchio/) を参照してください.

## What is URDF?
URDF (universal robot description format) はロボットのモデルを記述するフォーマットです.  
既成のURDFを`robotoc`で利用するだけであれば，URDFについての知識はほとんど必要ありません．   
本リポジトリの `urdf`フォルダに既成のURDFパッケージがあります．   
その中身を見れば分かりますが，URDFパッケージはurdf ファイル(`~.urdf`) とメッシュファイル (e.g., `~.dae` or `~.obj`)から成ります．   
urdf ファイルは他関節ロボットのツリー構造を記述し，メッシュファイルは可視化や衝突検証のために用いられます．   
より詳しく知りたい方はROSのチュートリアル http://wiki.ros.org/urdf/Tutorials などを参照してください.  

## Simplest example: ロボットマニピュレータ `iiwa14`

まずは最もシンプルな `robotoc.Robot` (`robotoc::Robot` in C++) の例としてロボットマニピュレータ（ロボットアーム）を扱います.  
ここで，C++での `robotoc::Robot` はPythonより非常に多くの機能があります.（Pythonでは最低限の機能しか提供していません．）

In [ ]:
import robotoc
path_to_urdf = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(path_to_urdf)

ロボットの様々な情報は以下のようにプリントできます．

In [ ]:
print(iiwa14)

プリントされた情報は以下の通りです．
- `dimq`: configuration（一般化座標）の次元
- `dimv`: 一般加速度の次元 (`dimq`と異なる場合があります)
- `dimu`: 入力トルクの次元
- `dim_passive`: passiveなジョイント（力をだせないジョイント）の次元． floating base を含みます．
- `frames`: ロボットのフレーム．評価関数や制約でエンドエフェクタを指定する際に用います．
- `joints`: ジョイントの情報です．
- `effort limit`: ジョイントのトルクの上限です．
- `velocity limit`: ジョイントの速度の上限です．
- `lower position limit`, `upper position limit`:　ジョイントの位置の上下限です．

これらのうち一部は以下のように個々に得られます．

In [ ]:
iiwa14.dimq()

In [ ]:
iiwa14.dimv()

In [ ]:
iiwa14.dimu()

In [ ]:
iiwa14.dim_passive()

In [ ]:
iiwa14.joint_effort_limit()

In [ ]:
iiwa14.joint_velocity_limit()

In [ ]:
iiwa14.lower_joint_position_limit()

In [ ]:
iiwa14.upper_joint_position_limit()

`robotoc`でロボットの可視化もできます．試しにランダムな関節角について可視化してみましょう．　　　   
（実際には，ランダムな関節角速度を生成し，それを積分して関節角軌道を生成しています．）

In [ ]:
import numpy as np
steps = 100
dt = 0.1
q = []
q.append(iiwa14.generate_feasible_configuration())
for i in range(steps):
    v = np.random.randn(iiwa14.dimv())
    q.append(iiwa14.integrate_configuration(q[-1], v, dt))

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf, viewer_type='meshcat')
viewer.display(dt, q)

Pythonでもシンプルな forward kinematics （指定した関節角に対する end-effector の位置計算） を試すことができます．

In [ ]:
end_effector_frame = 'iiwa_link_ee_kuka'
for e in q:
    iiwa14.forward_kinematics(e)
    print('q: ', e)
    print('frame position: ', iiwa14.frame_position(end_effector_frame))
    print('frame rotation: ', iiwa14.frame_rotation(end_effector_frame))
    print('frame placement (SE3): ', iiwa14.frame_placement(end_effector_frame))

## 4脚ロボット `ANYmal`

次に４脚ロボットを試してみます．    
4脚ロボットは**4つの点接触 (Point Contacts)**を伴う点がマニピュレータとの大きな違いです．   
接触は，接触フレームの名前かインデックスで指定できます．   
以下では名前で指定します．    

In [ ]:
contact_frames = ['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT']

また，接触の種類も指定する必要があります.   
4脚ロボットの接触は**点接触 (Point Contact)**ですので，以下のように指定します．

In [ ]:
contact_types = [robotoc.ContactType.PointContact for _ in contact_frames]

接触がある場合には， Baumgarte の安定化法のパラメータを設定する必要があります．  
これを適切に設定することで，正確な rigid contact を最適制御の中で再現します.    
詳細は [こちらの論文](https://www.researchgate.net/publication/234610391_A_Parametric_Study_on_the_Baumgarte_Stabilization_Method_for_Forward_Dynamics_of_Constrained_Multibody_Systems) などを参照して下さい.
適切なパラメータはMPCの離散化の時間ステップなどに依存します.    
手元で少し試して決めてください．   
私のこれまでの経験では，MPCの時間ステップの 2~10倍程度に Baumgarte　の安定化法の時間ステップを設定すればうまくいきます．   
ここでは 0.04とします．

In [ ]:
baumgarte_time_step = 0.04

いよいよロボットモデルを作成します．   
4脚ロボットは floating base を持つため，その点もコンストラクタで指定します．

In [ ]:
path_to_urdf = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
anymal = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                       contact_frames, contact_types, baumgarte_time_step)

ロボットモデルの詳細は以下で見れます．

In [ ]:
print(anymal)

再び，以下で可視化してみましょう．

In [ ]:
steps = 100
dt = 0.1
q = []
q.append(anymal.generate_feasible_configuration())
q[0][0:3] = np.zeros(3)
for i in range(steps):
    v = np.concatenate([0.1*np.random.randn(6), np.random.randn(anymal.dimu())])
    q.append(anymal.integrate_configuration(q[-1], v, dt))

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase,
                                        viewer_type='meshcat')
viewer.display(dt, q)

## ヒューマノイドロボット `iCub`

次に，ヒューマノイドロボットを扱います．   
ヒューマノイドロボットは**面接触 (Surface Contacts)** を2つ伴うため，以下のように設定します．

In [ ]:
contact_frames = ['l_sole', 'r_sole']
contact_types = [robotoc.ContactType.SurfaceContact for i in contact_frames]

baumgarte_time_step = 0.05

path_to_urdf = 'urdf/icub_description/urdf/icub.urdf'
icub = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                     contact_frames, contact_types, baumgarte_time_step)

ロボットモデルの詳細を見てみましょう．

In [ ]:
print(icub)

また，以下で可視化してみましょう．

In [ ]:
steps = 100
dt = 0.1
q = []
q.append(icub.generate_feasible_configuration())
q[0][0:3] = np.zeros(3)
for i in range(steps):
    v = np.concatenate([0.1*np.random.randn(6), np.random.randn(icub.dimu())])
    q.append(icub.integrate_configuration(q[-1], v, dt))

viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        base_joint_type=robotoc.BaseJointType.FloatingBase,
                                        viewer_type='meshcat')
viewer.display(dt, q)